In [20]:
import torch
import os
from flask import Flask, request, jsonify, render_template, flash, redirect
from werkzeug.utils import secure_filename

from transformers import ViTImageProcessor, ViTModel, AutoImageProcessor, AutoModelForImageClassification,  AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import requests
import torch
from datasets import load_dataset



In [11]:
device = "cuda" # the device to load the model onto

model_nlp = AutoModelForCausalLM.from_pretrained("stabilityai/stable-code-instruct-3b")
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-instruct-3b")

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.05it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:

# processor = ViTImageProcessor.from_pretrained('nateraw/food')
image_processor = AutoImageProcessor.from_pretrained('nateraw/food')
# model = ViTModel.from_pretrained('nateraw/food')
model_vision = AutoModelForImageClassification.from_pretrained('nateraw/food')

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [5]:
def get_top_k_pred_labels(im, k=3):

    inputs = image_processor(im, return_tensors="pt")
    with torch.no_grad():
        logits = model_vision(**inputs).logits
    # Get indices of the top two predicted labels
    top_k_values, top_k_indices = logits.topk(k, dim=-1)

    # Extract the indices as Python scalars
    predicted_labels = top_k_indices.tolist()

    predicted_labels_with_labels = [[model_vision.config.id2label[label] for label in labels] for labels in predicted_labels]

    return predicted_labels_with_labels[0]

In [6]:
# TESTING VIT 
filename = "Test/pizza_slice.jpg"
with Image.open(filename) as image:
    image.load()
predicted_labels_with_labels = get_top_k_pred_labels(image)
predicted_labels_with_labels

['pizza', 'bruschetta', 'garlic_bread']

In [7]:
def get_nlp_op_old(messages):

    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

    model_inputs = encodeds.to(device)
    model_nlp.to(device)

    generated_ids = model_nlp.generate(model_inputs, max_new_tokens=1000, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    print(decoded[0])
    return decoded

In [12]:
def get_nlp_op(messages):
    prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
    inputs = tokenizer([prompt], return_tensors="pt").to(model_nlp.device)

    tokens = model_nlp.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.5,
        top_p=0.95,
        top_k=100,
        do_sample=True,
        use_cache=True
    )

    output = tokenizer.batch_decode(tokens[:, inputs.input_ids.shape[-1]:], skip_special_tokens=False)[0]
    return output

In [13]:
message_template = [
       {
        "role": "system",
        "content": "You are a helpful and polite assistant for diabetes patients",
    }] 

In [14]:
# TESTING NLP Model

test_msg = message_template + [{"role": "user", "content": "Is pizza good for me as a diabetes patient"}]
    
get_nlp_op(test_msg)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


"Pizza is a delicious and convenient food choice for many people, including those with diabetes. The key thing to remember when eating pizza is to monitor portions and make sure to include healthy sources of protein and complex carbohydrates. Be aware that some toppings, such as fried dough or tomato sauce, may be high in fat and sugars and could increase your risk of developing diabetes. It's best to consult with your doctor before eating any food to ensure you are getting the right nutrition for your health.<|im_end|>\n<|endoftext|>"

In [38]:
app = Flask(__name__)
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = set(['jpg', 'jpeg'])

def allowed_file(filename):     
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route("/")
def showHomePage():
      # response from the server
    return render_template('index.html')

@app.route("/food_detector", methods = ["GET", "POST"])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            print('No file part')
            return redirect(request.url)
        file = request.files['file']

        if file.filename == '':
            print('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            
            filename = secure_filename(file.filename)
            print("the allowed filename = " + str(filename))
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
    return render_template('index.html')

@app.route('/chatbot', methods=['POST'])
def chatbot():
    data = request.json
    print("data = " + str(data))
    user_input = data['message']
    print("user_ip = " + str(user_input))
    # Generate response
    user_msg = message_template + [{"role": "user", "content":user_input}]
    response = get_nlp_op(user_msg)
    print(response)
    return render_template('index.html')






In [39]:
if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


data = {'message': 'what is diabetes'}
user_ip = what is diabetes


127.0.0.1 - - [02/Apr/2024 11:22:10] "POST /chatbot HTTP/1.1" 200 -


Diabetes is a chronic condition that affects how the body processes food and drinks to produce energy.Get a detailed explanation about diabetes here.<|im_end|>
<|endoftext|>
